<a href="https://colab.research.google.com/github/denskoy/dolzhenkov_task/blob/main/Meteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
from pyspark.sql.functions import desc, asc, month

spark = SparkSession.builder \
        .appName("meteo") \
        .getOrCreate()

schema = StructType([
    StructField("station_id", StringType(), True),
    StructField("date", DateType(), True),
    StructField("temperature", FloatType(), True),
    StructField("precipitation", FloatType(), True),
    StructField("wind_speed", FloatType(), True)
])
df = spark.read.csv("/content/weather_data.csv", header=True, schema=schema, sep=",")
df.printSchema()
#Найдите топ-5 самых жарких дней за все время наблюдений.
temp_days = df.orderBy(df.temperature.desc())
temp_days.select("date", "temperature").limit(5).show()
#Найдите метеостанцию с наибольшим количеством осадков за последний год.
prec_station = df.where("date >= '2023-01-01' and date <= '2023-12-31'").groupBy("station_id").agg({"precipitation": "sum"})
max_prec = prec_station.orderBy(desc("sum(precipitation)")).limit(1).show()
# Средняя температура по месяцам
month_avg = df.withColumn("month", month("date")).groupBy("month").avg("temperature").orderBy("month")
month_avg.show()
spark.stop

root
 |-- station_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: float (nullable = true)
 |-- precipitation: float (nullable = true)
 |-- wind_speed: float (nullable = true)

+----------+-----------+
|      date|temperature|
+----------+-----------+
|2021-08-20|   39.98283|
|2023-12-02|  39.967976|
|2022-03-28|  39.824688|
|2019-02-11|  39.767376|
|2020-06-10|   39.69148|
+----------+-----------+

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302670955658|
+----------+------------------+

+-----+------------------+
|month|  avg(temperature)|
+-----+------------------+
|    1|11.356518470007797|
|    2|  9.06722993850708|
|    3| 7.244080115937525|
|    4|12.024529053670603|
|    5| 9.902883278588726|
|    6|13.421092370936744|
|    7| 6.185718382110021|
|    8|10.967800280712183|
|    9| 9.596744181906304|
|   10| 9.098843419781097|
|   11| 7.265890174138714|
|   12|11.218592057529005|

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x791d91cdded0>>